In [8]:
'''Propagation from one layer to the next. '''
import numpy as np

def fft(A):
    '''exp{-i2pi (lu + mv)}'''
    return np.fft.fftshift(np.fft.fftn(np.ftt.ifftshift(A)))

def ifft(A):
    '''exp{i2pi (lu + mv)}'''
    return np.fft.fftshift(np.fft.ifftn(np.fft.ifftshift(A)))

class Propagation(object):
    def __init__(self,w):
        self.w = w
    def setAperture(self,U,u,v):
        self.U = U
        
        self.A = fft(U)

        self.u = u
        self.v = v
    def R2propKernel(self,UVmax2):
        D = UVmax2/self.w**2
        pa = 1
        pb = D+1
        return 2.*np.pi*self.w*3./16.*(-2./3.*(D+1)**2* (pb**(-3./2.) - pa**(-3./2.)) + 4.*(D+1)*(pb**(-1./2.) - pa**(-1./2.)) + 2.*(pb**(1./2.) - pa**(1./2.)))
    def solveAperturePartition(self):
        '''Find UVmax such that lagrange remainder in phase is less than 1 radian.'''
        a0 = 2*np.pi*self.w
        p = [a0/8., 0., -a0*3./4., a0, -a0*3./8. - 1]
        roots = np.roots(p)
        i = 0
        while i < 4:
            if np.imag(roots[i]) == 0 and np.real(roots[i]) > 0:
                return np.real(np.sqrt(self.w**2*(roots[i]**2 - 1)))
            i += 1
        return np.real(np.sqrt(self.w**2*(roots[1]**2 - 1))) #typically second root when solved by numpy
         
    def plotR2propKernel(self):
        import pylab as plt
        sol = self.solveAperturePartition()

        UVmax = np.linspace(10,1000,100000)
        r2 = self.R2propKernel(UVmax**2)
        plt.plot(UVmax,r2)
        plt.plot([sol,sol],[np.min(r2),1.],ls='--',c='black')
        plt.plot([np.min(UVmax),sol],[1.,1.],ls='--',c='black')
        plt.xlabel('Aperture size (lambda)')
        plt.ylabel('Phase error w={0}'.format(self.w))
        plt.yscale('log')
        plt.xscale('log')
        plt.show()
       
    def propagate(self):
        '''Piecewise propagate'''
        UVmax = self.solveAperturePartition()
        Nu = np.ceil((self.u[-1] - self.u[0])/UVmax*2 - 1)#minimum pieces with overlap
        Nv = np.ceil((self.v[-1] - self.v[0])/UVmax*2 - 1)
        u0 = np.linspace(self.u[0],self.u[-1],int(Nu))
        v0 = np.linspace(self.v[0],self.v[-1],int(Nv))
        #coords of recieving aperture
        U,V = np.meshgrid(self.u,self.v)
        Uprop = np.zeros_like(U)
        #Fourier parameters
        l = np.fft.fftshift(np.fft.fftfreq(len(self.u),d = np.abs(self.u[1] - self.u[0])))
        m = np.fft.fftshift(np.fft.fftfreq(len(self.v),d = np.abs(self.v[1] - self.v[0])))
        L,M = np.meshgrid(l,m)
    
        propKernel1 = np.exp(1j*2.*np.pi*self.w*(1 + (U**2 + V**2)/self.w**2))
        propKernel2fourier = 1j/2.*self.w*np.exp(-1j*2.*np.pi/self.w*(U**2 + V**2 + self.w*(L*U + M*V) + (L**2 + M**2)*self.w**2/4.))
        plt.imshow(np.angle(propKernel2fourier))
        plt.colorbar()
        plt.show()
        #weight factor to do with R2, angular powerspectrum, prop
        Uprop = -1j*propKernel1*ifft(self.A*propKernel2fourier)
        return Uprop
                
                

if __name__=='__main__':
    #P = Propagation(100)
    #P.plotR2propKernel()
    import pylab as plt
    sols = []
    for w in np.linspace(10,100000,250):
        P = Propagation(w)
        sols.append(P.solveAperturePartition())
        print("w {0} -> uvmax {1}".format(w,sols[-1]))
    plt.plot(sols,np.linspace(10,100000,250))
    plt.xlabel('Aperture size (lambda)')
    plt.ylabel(r'Minimum $w$')
    plt.show()
    P = Propagation(100)
    M = 1
    x0 = np.random.uniform(low=-50,high=50,size=M)
    y0 = np.random.uniform(low=-50,high=50,size=M)
    p0 = np.random.uniform(size=M)

    wavelength = 1.
    N=100
    print('number:{0}'.format(N))
    x = np.linspace(-100,100,N)
    dx = np.abs(x[1]-x[0])
    X,Y = np.meshgrid(x,x)
    Usky = np.zeros([N,N])
    for xi,yi,pi in zip(x0,y0,p0):
        Usky += pi*np.exp(-((X-xi)**2 + (Y-yi)**2)/(2*dx)**2)
    P.setAperture(Usky,x,x)
    Uprop = P.propagate()
        

w 10.0 -> uvmax 8.449143434407812
w 411.56626506024094 -> uvmax 179.63846228411268
w 813.1325301204819 -> uvmax 315.6762360708003
w 1214.6987951807228 -> uvmax 440.25879882285363
w 1616.2650602409637 -> uvmax 557.9230683222033
w 2017.8313253012047 -> uvmax 670.6929795619278
w 2419.3975903614455 -> uvmax 779.7089196258847
w 2820.9638554216867 -> uvmax 885.6986982963941
w 3222.5301204819275 -> uvmax 989.1649809062826
w 3624.0963855421683 -> uvmax 1090.4745312515604
w 4025.6626506024095 -> uvmax 1189.906020946743
w 4427.22891566265 -> uvmax 1287.6778962334251
w 4828.795180722891 -> uvmax 1383.965688754351
w 5230.361445783132 -> uvmax 1478.9133194593674
w 5631.9277108433735 -> uvmax 1572.6407833403498
w 6033.493975903614 -> uvmax 1665.2495500649993
w 6435.060240963855 -> uvmax 1756.82646678546
w 6836.626506024096 -> uvmax 1847.4466461511245
w 7238.1927710843365 -> uvmax 1937.1756477450872
w 7639.759036144578 -> uvmax 2026.0711553857611
w 8041.325301204819 -> uvmax 2114.184287236636
w 8442.

AttributeError: module 'numpy' has no attribute 'ftt'

In [54]:
plt.imshow(np.abs(Uprop))
plt.show()